# In-Context Learning


In-context learning is a generalisation of few-shot learning where the LLM is provided a context as part of the prompt and asked to respond by utilising the information in the context.

* Example: *"Summarize this research article into one paragraph highlighting its strengths and weaknesses: [insert article text]”*
* Example: *"Extract all the quotes from this text and organize them in alphabetical order: [insert text]”*

A very popular technique that you will learn in week 5 called Retrieval-Augmented Generation (RAG) is a form of in-context learning, where:
* a search engine is used to retrieve some relevant information
* that information is then provided to the LLM as context


In this example we download some recent research papers from arXiv papers, extract the text from the PDF files and ask Gemini to summarize the articles as well as provide the main strengths and weaknesses of the papers. Finally we print the summaries to a local html file and as markdown.

In [1]:
!pip install requests bs4 google-generativeai pypdf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.6/329.6 kB 7.1 MB/s eta 0:00:00


In [2]:
import os
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai
from urllib.request import urlopen, urlretrieve
from IPython.display import Markdown, display
from pypdf import PdfReader
from datetime import date
from tqdm import tqdm
from google.colab import userdata

In [ ]:
API_KEY = "AIzaSyA0mXFee40Cyg7lIK4NbOP308-49"
genai.configure(api_key=API_KEY)

We select those papers that have been featured in Hugging Face papers.

In [3]:
BASE_URL = "https://huggingface.co/papers"
page = requests.get(BASE_URL)
soup = BeautifulSoup(page.content, "html.parser")
h3s = soup.find_all("h3")

papers = []

for h3 in h3s:
    a = h3.find("a")
    title = a.text
    link = a["href"].replace('/papers', '')

    papers.append({"title": title, "url": f"https://arxiv.org/pdf{link}"})

Code to extract text from PDFs.

In [4]:
def extract_paper(url):
    html = urlopen(url).read()
    soup = BeautifulSoup(html, features="html.parser")

    # kill all script and style elements
    for script in soup(["script", "style"]):
        script.extract()    # rip it out

    # get text
    text = soup.get_text()

    # break into lines and remove leading and trailing space on each
    lines = (line.strip() for line in text.splitlines())
    # break multi-headlines into a line each
    chunks = (phrase.strip() for line in lines for phrase in line.split("  "))
    # drop blank lines
    text = '\n'.join(chunk for chunk in chunks if chunk)

    return text


def extract_pdf(url):
    pdf = urlretrieve(url, "pdf_file.pdf")
    reader = PdfReader("pdf_file.pdf")
    text = ""
    for page in reader.pages:
        text += page.extract_text() + "\n"
    return text


def printmd(string):
    display(Markdown(string))

In [ ]:
LLM = "gemini-2.5-flash"
model = genai.GenerativeModel(LLM)

We use Gemini to summarize the papers.

In [5]:
PROMPT_TEMPLATE = """
You are given the full text of a research paper.

Produce a single Markdown table with exactly three columns:

| Summary | Strengths | Weaknesses |

Guidelines:
- The Summary column should contain a concise one-paragraph summary of the paper.
- The Strengths and Weaknesses columns should contain short, clear analytical points.
- All content must be inside ONE Markdown table.
- Do NOT include any text before or after the table.
- Use valid Markdown table syntax.
- Base your analysis only on the given paper.

Paper text:
"""

In [7]:
for paper in tqdm(papers):
    try:
      paper_text = extract_pdf(paper["url"])
      paper["summary"] = model.generate_content(PROMPT_TEMPLATE + paper_text).text

    except:
      print("Generation failed")
      paper["summary"] = "Paper not available"

 75%|███████▌  | 15/20 [05:54<01:53, 22.68s/it]

Generation failed


100%|██████████| 20/20 [07:26<00:00, 22.30s/it]


We print the results to a html file.

In [10]:
!pip install markdown

In [11]:
import markdown

In [12]:
page = f"<html> <head> <h1>Daily Dose of AI Research</h1> <h4>{date.today()}</h4> <p><i>Summaries generated with: {LLM}</i>"
with open("papers.html", "w") as f:
    f.write(page)
for paper in papers:
    # Modify the codes to show the table
    html_summary = markdown.markdown(
        paper["summary"],
        extensions=["tables"]
    )

    page = f"""
    <h2><a href="{paper["url"]}">{paper["title"]}</a></h2>
    {html_summary}
    """

    with open("papers.html", "a") as f:
        f.write(page)
end = "</head>  </html>"
with open("papers.html", "a") as f:
    f.write(end)

We can also print the results to this notebook as markdown.

In [8]:
# Modify the code to show the table
for paper in papers:
    printmd(
        f"## [{paper['title']}]({paper['url']})\n\n"
        f"{paper['summary']}\n\n---\n"
    )

## [mHC: Manifold-Constrained Hyper-Connections](https://arxiv.org/pdf/2512.24880)

| Summary | Strengths | Weaknesses |
|---|---|---|
| This paper introduces Manifold-Constrained Hyper-Connections (mHC), an extension of Hyper-Connections (HC) designed to address HC's training instability, restricted scalability, and memory overhead. mHC projects HC's residual connection matrices onto a specific manifold (doubly stochastic matrices using Sinkhorn-Knopp) to restore the identity mapping property, ensuring signal stability. It also incorporates rigorous infrastructure optimizations like kernel fusion, recomputing, and communication overlapping for efficiency. Empirical results demonstrate mHC's superior stability, scalability, and performance gains in large-scale language model pre-training with marginal overhead. | Restores identity mapping property, significantly improving training stability and mitigating signal explosion/vanishing. Achieves tangible performance improvements and superior scalability compared to both baseline and unconstrained HC. Incorporates rigorous infrastructure optimizations (kernel fusion, recomputing, communication overlapping) to ensure efficiency with minimal overhead. Benefits from strong theoretical properties of doubly stochastic matrices, such as norm preservation and compositional closure. | The Sinkhorn-Knopp algorithm provides an approximate solution, leading to slight deviations from perfect double stochasticity, especially in composite mappings. Introduces additional architectural and implementation complexity compared to standard residual connections. Despite optimizations, there is still a reported 6.7% additional time overhead for an expansion rate of n=4. The current manifold choice (doubly stochastic matrices) might not be universally optimal, and other constraints could be explored. |

---


## [Youtu-LLM: Unlocking the Native Agentic Potential for Lightweight Large Language Models](https://arxiv.org/pdf/2512.24618)

| Summary | Strengths | Weaknesses |
|---|---|---|
| Youtu-LLM is a 1.96B-parameter language model pre-trained from scratch to achieve high computational efficiency and native agentic intelligence. It features a compact MLA architecture with 128k context, a STEM-oriented tokenizer, and a multi-stage "Commonsense-STEM-Agent" curriculum. Its scalable agentic mid-training, utilizing diverse trajectory data, cultivates intrinsic planning and reflection. Evaluations show Youtu-LLM sets new state-of-the-art for sub-2B LLMs, outperforming or rivaling larger models on general and agent-specific benchmarks. | Achieves strong native agentic capabilities through systematic pre-training, not just post-hoc methods. Highly computationally efficient (1.96B parameters) with long-context support (128k) for resource-constrained use. Employs a comprehensive multi-stage training curriculum and high-quality, diverse agentic trajectory data. Demonstrates state-of-the-art performance for sub-2B models, often surpassing larger LLMs on agentic tasks. Utilizes robust training methodologies including two-stage SFT and refined RL for stability and performance. | Agentic capabilities still lag behind very large proprietary LLMs due to computational resource limitations. Long reasoning trajectories can lead to increased inference latency, impacting efficiency. Current scope is limited to text-based environments, lacking multimodal perception. Absence of suitable agentic mathematical benchmarks for instruction-tuned models hinders comprehensive evaluation. Potential for performance degradation if domain-specific trajectory data is excessively upsampled, indicating a need for careful data balancing. |

---


## [Let It Flow: Agentic Crafting on Rock and Roll, Building the ROME Model within an Open Agentic Learning Ecosystem](https://arxiv.org/pdf/2512.24873)

| Summary | Strengths | Weaknesses |
|---|---|---|
| This paper introduces the Agentic Learning Ecosystem (ALE), a comprehensive infrastructure comprising ROLL (RL training), ROCK (sandboxed environments), and iFlow CLI (agent framework) designed to streamline the development and deployment of agentic LLMs for complex, multi-turn tasks. It presents ROME, an open-source agent LLM trained within ALE using a novel Interaction-Perceptive Agentic Policy Optimization (IPA) algorithm and rigorously curated data, achieving strong performance on various agentic benchmarks and demonstrating production readiness. | - Provides a complete, end-to-end open-source ecosystem (ALE) for agentic LLM development and deployment.<br>- Introduces IPA, a novel RL algorithm that improves training stability and efficiency for long-horizon tasks by optimizing over semantic interaction chunks.<br>- Employs a robust, multi-tiered data composition strategy with strong verification and safety alignment.<br>- ROME achieves competitive performance on diverse agentic benchmarks, often outperforming similarly sized models and rivaling larger ones.<br>- Designed for production deployment, demonstrated by ROME's successful integration and real-world validation.<br>- Introduces Terminal Bench Pro, a more rigorous benchmark for terminal agents.<br>- Explicitly addresses and integrates safety-aligned data to mitigate general-security issues in agent behavior. | - ROME's performance on the most challenging benchmark (Terminal Bench Pro) remains limited, indicating significant room for improvement in complex real-world tasks.<br>- The comprehensive ecosystem might present a steep learning curve and high setup complexity for new users.<br>- Heavy reliance on synthesized data, despite verification, could introduce biases or limit real-world generalizability.<br>- The extensive data curation and filtering process, while beneficial, could potentially lead to overfitting on benchmark characteristics.<br>- The computational resources required for training on "over one million trajectories" and complex RL might be substantial, limiting accessibility.<br>- The systematic investigation of safety measures is noted as future work, suggesting current safety protocols may not be fully comprehensive or generalized. |

---


## [GaMO: Geometry-aware Multi-view Diffusion Outpainting for Sparse-View 3D Reconstruction](https://arxiv.org/pdf/2512.25073)

| Summary | Strengths | Weaknesses |
|---|---|---|
| GaMO (Geometry-aware Multi-view Diffusion Outpainter) is a novel framework for sparse-view 3D reconstruction that reformulates the problem as multi-view outpainting rather than novel view generation. By expanding the field of view of existing input images using a zero-shot multi-view diffusion model conditioned on geometry priors, GaMO effectively mitigates common artifacts like holes, ghosting, and geometric inconsistencies. The method achieves state-of-the-art reconstruction quality on Replica and ScanNet++ datasets with significant speed improvements (25x faster) over previous diffusion-based approaches. | - Effectively mitigates holes, ghosting, and geometric inconsistencies in sparse-view 3D reconstruction. <br> - Achieves state-of-the-art performance across multiple metrics (PSNR, SSIM, LPIPS, FID) on benchmark datasets. <br> - Significantly faster (25x speedup, under 10 minutes) compared to prior diffusion-based methods. <br> - Operates in a zero-shot manner, requiring no fine-tuning of the diffusion backbone. <br> - Preserves geometric consistency by expanding existing views' FOV rather than generating new viewpoints. <br> - Demonstrates strong generalization to diverse scenes, including large-scale 360-degree environments. | - Unable to recover content that is completely occluded from all input views. <br> - Performance is sensitive to the distribution and alignment of input views. <br> - Heavily occluded regions remain challenging for reconstruction. |

---


## [A unified framework for detecting point and collective anomalies in operating system logs via collaborative transformers](https://arxiv.org/pdf/2512.23380)

| Summary | Strengths | Weaknesses |
|---|---|---|
| CoLog is a novel framework for operating system log anomaly detection that addresses the limitations of existing unimodal and multimodal approaches. It formulates log anomaly detection as a multimodal sentiment analysis problem, enabling the unified detection of both point and collective anomalies. CoLog achieves this by collaboratively encoding semantic and sequence log modalities using a collaborative transformer architecture with multi-head impressed attention to capture nuanced interactions, and a modality adaptation layer to handle data heterogeneity. The framework demonstrates superior performance, achieving high precision, recall, and F1 scores across multiple benchmark datasets, and its implementation is publicly available. | - Unified framework capable of detecting both point and collective anomalies. <br> - Novel architecture combining collaborative transformers, multi-head impressed attention, and a modality adaptation layer. <br> - Achieves state-of-the-art performance with high accuracy, precision, recall, and F1-scores across diverse benchmark datasets. <br> - Demonstrates strong generalization and robustness to unseen log events and new datasets. <br> - Effectively handles class imbalance using the Tomek link technique. <br> - Open-source implementation promotes reproducibility and further research. <br> - Meaningful fusion of modalities in a shared latent space. | - Evaluated in batch-processing mode; real-time performance regarding latency and computational resources is not fully explored. <br> - Initial interpretation and labeling of complex, human-unreadable OS log entries can be challenging. <br> - Requires continual adaptation and retraining to maintain validity with evolving log structures and system updates. <br> - Inference time is higher compared to some other deep learning models like LSTM or CNN. <br> - Performance may degrade in extreme class imbalance scenarios (e.g., anomaly ratio <1%) or with rapidly evolving log templates. |

---


...

## [BEDA: Belief Estimation as Probabilistic Constraints for Performing Strategic Dialogue Acts](https://arxiv.org/pdf/2512.24885)

| Summary | Strengths | Weaknesses |
|---|---|---|
| The paper introduces BEDA (Belief Estimation for Dialogue Acts), a framework that bridges belief estimation and utterance generation in strategic dialogue by operationalizing dialogue acts as probabilistic constraints. BEDA formalizes two core dialogue acts, Adversarial and Alignment, and consists of a World Set, a Belief Estimator, and a Conditional Generator. Evaluated across competitive (Conditional Keeper–Burglar), cooperative (Mutual Friends), and negotiation (CaSiNo) settings, BEDA consistently outperforms strong baselines, demonstrating significant improvements in success rates and negotiation outcomes by effectively using inferred beliefs to constrain generation. | 1. **Novel Framework:** Bridges the gap between belief estimation and dialogue act generation by using beliefs as probabilistic constraints. <br> 2. **Strong Theoretical Foundation:** Mathematically defines Adversarial and Alignment Dialogue Acts based on rigorous belief and common knowledge formulations. <br> 3. **Empirical Superiority:** Consistently outperforms strong baselines (e.g., CoT, Self-Reflect, MindDial) across diverse dialogue tasks and various LLM backbones (GPT-3.5, GPT-4, LLaMA, Qwen). <br> 4. **Generalizability:** Effective in competitive, cooperative, and negotiation scenarios, demonstrating robustness. <br> 5. **Efficiency Improvements:** Achieves higher success rates with fewer turns and tokens in cooperative tasks (Mutual Friends). <br> 6. **Modularity:** Separates belief estimation (using a lightweight encoder) from LLM generation, reducing computational costs while maintaining performance. <br> 7. **Mitigates Hallucinations:** Reduces common LLM hallucinations like friend-list comparison and looping dialogues by introducing belief-state constraints. | 1. **Fixed World Set:** The current framework relies on a predefined World Set, limiting its adaptability to dynamic or evolving dialogue contexts. <br> 2. **Coarse Dialogue Act Granularity:** Only two broad dialogue act categories (Adversarial, Alignment) are formalized, potentially overlooking finer-grained strategic nuances (e.g., Concession, Hedging). <br> 3. **Lower Accuracy on Complex Tasks:** Belief estimation accuracy is notably lower on the CaSiNo negotiation task compared to simpler synthetic datasets, suggesting challenges with multi-class belief structures. <br> 4. **LLM Backbone Sensitivity:** Weaker LLMs (e.g., LLaMA without fine-tuning) struggle significantly with certain tasks (like Mutual Friends), leading to poor performance and hallucinations, indicating a reliance on strong base models. <br> 5. **Format Errors:** LLMs still exhibit format errors in generated outputs, requiring manual exclusion of invalid responses. |

---


## [Forging Spatial Intelligence: A Roadmap of Multi-Modal Data Pre-Training for Autonomous Systems](https://arxiv.org/pdf/2512.24385)

| Summary | Strengths | Weaknesses |
|---|---|---|
| This paper provides a comprehensive roadmap for multi-modal data pre-training aimed at achieving "Spatial Intelligence" in autonomous systems. It categorizes pre-training paradigms from single-modality to unified frameworks, analyzes platform-specific datasets across vehicles, drones, and other robots, and explores applications like 3D object detection, semantic occupancy prediction, and open-world perception/planning. The work emphasizes the pivotal role of foundation models, generative world models, and Vision-Language-Action (VLA) architectures, while also identifying critical challenges and future research directions for robust real-world deployment. | - Comprehensive and well-structured taxonomy of pre-training methods, datasets, and applications.<br>- Strong emphasis on the role of foundation models and generative objectives for holistic scene understanding.<br>- Highlights effective strategies for data efficiency, such as cross-modal distillation and self-supervised learning.<br>- Provides a clear, forward-looking roadmap with critical challenges and promising future research directions.<br>- Covers diverse autonomous platforms, showcasing broad applicability of the concepts.<br>- Empirical analysis supports the efficacy of discussed pre-training paradigms on key benchmarks. | - Primarily a high-level survey, lacking in-depth technical details of individual algorithms.<br>- The rapidly evolving nature of the field means some "future directions" may quickly become current or outdated.<br>- Limited discussion on the ethical, societal, or regulatory implications of advanced autonomous systems.<br>- While touching on planning, the focus remains heavily on perception aspects of spatial intelligence.<br>- Does not offer concrete solutions or detailed research avenues for real-time inference challenges of large foundation models. |

---


## [Figure It Out: Improving the Frontier of Reasoning with Active Visual Thinking](https://arxiv.org/pdf/2512.24297)

| Summary | Strengths | Weaknesses |
|---|---|---|
| This paper introduces FIGR, a novel framework that enhances complex reasoning in large language models by integrating "active visual thinking" through end-to-end reinforcement learning. FIGR generates executable code to construct and iteratively refine visual representations (figures) during multi-turn problem-solving, providing dynamic and geometrically consistent feedback. An adaptive reward mechanism guides the model to selectively invoke visual reasoning, leading to significant performance improvements on challenging mathematical benchmarks like AIME and BeyondAIME, by enabling more stable and coherent reasoning over global structural properties. | Generates geometrically consistent and interpretable visual feedback via executable code. Adaptive reward mechanism promotes selective and efficient use of visual thinking. End-to-end reinforcement learning enables autonomous learning of visual invocation strategies without supervised fine-tuning. Achieves substantial performance gains on complex mathematical reasoning tasks, outperforming text-only and other multimodal baselines. Addresses limitations of noisy image generation and predefined tool operations in prior multimodal models. | Relies on a separate LLM for "suitability" classification, introducing an external dependency. Computational cost of reinforcement learning with multi-turn rollouts and code execution can be high. Dependency on a sandboxed code interpreter introduces potential failure points for code execution. Generalizability of code generation for highly novel or abstract visual representations might be challenging. Primarily demonstrated on mathematical reasoning, with broader applicability implied but not extensively shown. |

---


## [Factorized Learning for Temporally Grounded Video-Language Models](https://arxiv.org/pdf/2512.24097)

| Summary | Strengths | Weaknesses |
|---|---|---|
| This paper introduces D2VLM, a novel framework for temporally grounded video-language models that addresses limitations in existing approaches by factorizing the learning of temporal grounding and textual response. D2VLM adopts a "grounding then answering with evidence referencing" paradigm, utilizing "evidence tokens" to explicitly capture event-level visual semantics. It also proposes Factorized Preference Optimization (FPO) and a synthetic dataset with factorized perturbations to enhance learning for both tasks. Experiments demonstrate D2VLM's superior performance across various video understanding benchmarks. | <ul><li>Decouples temporal grounding and textual response, addressing a key limitation of coupled learning objectives.</li><li>Introduces novel "evidence tokens" that capture event-level visual semantics, providing rich context for text generation.</li><li>Proposes Factorized Preference Optimization (FPO) to explicitly optimize for both temporal grounding and textual response.</li><li>Develops a factorized synthetic dataset to enable controlled and scalable preference learning for explicit temporal grounding.</li><li>Achieves state-of-the-art performance across various temporal grounding and dense captioning tasks, often with a smaller model size.</li><li>Incorporates an explicit consistency constraint between grounding and answering stages, enhancing logical coherence.</li></ul> | <ul><li>Synthetic data generation primarily focuses on negative samples, limiting diversity in positive preference data.</li><li>Performance on some tasks, like episodic memory and YouCook2 dense video captioning, still has significant room for improvement.</li><li>General video QA performance, while improved, is noted to lag behind SOTA due to lack of large-scale generic pretraining and smaller model size.</li><li>The framework's multi-component nature (D2VLM, evidence tokens, FPO, synthetic data) might increase implementation complexity.</li><li>Relies on an off-the-shelf LLM for textual perturbations, which could introduce external biases.</li></ul> |

---


## [Valori: A Deterministic Memory Substrate for AI Systems](https://arxiv.org/pdf/2512.22280)

| Summary | Strengths | Weaknesses |
|---|---|---|
| Valori is a deterministic AI memory substrate designed to eliminate non-determinism in vector embedding storage and retrieval, which typically arises from floating-point arithmetic differences across hardware. It achieves this by replacing floating-point operations with fixed-point arithmetic (Q16.16) and modeling memory as a replayable state machine. Valori guarantees bit-identical memory states, snapshots, and search results across diverse platforms, enabling reproducible, auditable, and trustworthy AI systems, particularly for safety-critical and regulated applications. | Guarantees bit-identical memory states, snapshots, and search results across different hardware architectures (x86, ARM, RISC-V, WASM). <br> Enables full replayability and auditability for AI systems, crucial for regulated industries and safety-critical applications. <br> Achieves high semantic fidelity (99.8% Recall@10) despite using fixed-point quantization, demonstrating practical applicability. <br> Implements deterministic indexing structures by removing stochasticity from HNSW graph construction. <br> Offers configurable precision layers as a memory contract, allowing trade-offs between performance and precision without sacrificing determinism. <br> Open-source reference implementation is available. | Software-based fixed-point arithmetic can introduce performance overhead compared to hardware-accelerated floating-point operations. <br> Q16.16 fixed-point format has a limited dynamic range, potentially leading to quantization errors for extreme vector component values. <br> Valori's determinism boundary is at the memory layer; it does not address non-determinism originating from the neural network inference (embedding generation) itself. <br> The evaluation of semantic fidelity is based on a specific model and dataset, and universal recall preservation across all models/distributions is not claimed. <br> Some performance optimizations (e.g., SIMD integer instructions) and higher precision formats are noted as future work, implying current limitations. |

---
